![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Introduction

This notebook demonstrates how to use the `trend_scanning_labels` method from MlFinLab to predict the current trend direction of the BTCUSD pair.

In [1]:
import plotly.graph_objects as go
from matplotlib.colors import ListedColormap

from trend_scanning_labels import trend_scanning_labels

# Get asset prices.
qb = QuantBook()
symbol = qb.add_crypto("BTCUSD", Resolution.DAILY).symbol
history = qb.history(symbol, datetime(2016, 1, 1), datetime(2024, 1, 1))
closes = history.loc[symbol]['close']

# Get trend labels.
trend = trend_scanning_labels(
    closes, closes.index, observation_window=20, look_forward=False, 
    min_sample_length=5
)['bin'].dropna()

# Calculate the accuracy.
correct_predictions = np.sign(
    closes.pct_change().shift(-1).loc[trend.index].iloc[:-1]
) == np.sign(trend.iloc[:-1])
accuracy = sum(correct_predictions) / len(correct_predictions)

# Plot the results.
go.Figure(
    go.Scatter(
        x=trend.index, y=np.log10(closes), mode='markers',
        marker=dict(color=trend, size=4, colorscale=['red', 'blue'])
    ),
    dict(
        title="BTCUSD Trend Labels<br><sup>The daily trend predictions "
            + f"were correct {round(accuracy*100, 1)}% of the time</sup>",
        xaxis_title="Date", yaxis_title="Log Price"
    )
).show()

# Simulate a Backtest

Now that you have the predictions of the current trend for each day, simulate a rough backtest that holds Bitcoin when the predicted trend is up and holds USD when it's not.

In [2]:
from backtestlib import rough_daily_backtest

portfolio_weights = pd.DataFrame(trend.where(trend != -1, 0))
portfolio_weights.columns = [symbol]
rough_daily_backtest(qb, portfolio_weights)